In [1]:
# But first, let's install the dependecies.
!pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.2 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
from video_prompter import get_connection

# TODO: setup .env file
load_dotenv()

# connect to VideoDB
conn = get_connection()

### ----- Upload a fresh video --------- #####
def fresh_video(url):
    video = conn.upload(url=url)
    #index spoken content in the video
    video.index_spoken_words()
    return video

#### ------ run the prompt on video --------####
def videodb_prompter(video_id, prompt):
    video = get_video(video_id)
    #get all the segment of videos that are
    return video_prompter(video, prompt)

In [3]:
from videodb import play_stream

# ----Existing video case ----
# video_id = "null"


# ---- Fresh video case ----
url = "https://youtu.be/pAMy7IhOVQE?si=6HMxVGP13hAHtmW3"
video = fresh_video(url)
video_id = video.id


#watch the original video
video = conn.get_collection().get_video(video_id)
video.player_url

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/71ad9a7c-6b05-46f8-8248-ed314cc94e9f.m3u8'

In [29]:
# Create new scene index
from videodb import IndexType
from videodb import SceneExtractionType
 
index_id = video.index_scenes(
    extraction_type=SceneExtractionType.time_based,
    extraction_config={"time":1, "select_frames": ['middle']},
    prompt="Describe in detail the actions, gestures, and movements of each character visible in the frame. Include their position, facial expressions, and any interactions with objects or other characters.",
    name="Scene Index of actions"
)


In [ ]:
scene = video.get_scene_index(index_id)

In [ ]:
from video_prompter import scene_prompter


user_prompt = "talking on phone"
result = scene_prompter(scene, user_prompt)


In [ ]:
# search matching video segments and watch the stream
from videodb import SearchType
from videodb import IndexType
from videodb.asset import VideoAsset, AudioAsset, ImageAsset
from videodb.timeline import Timeline
from video_prompter import extract_descriptions

timeline = Timeline(conn)
for description in extract_descriptions(result):
    search_res = video.search(description, scene_index_id = index_id, index_type = IndexType.scene)
    matched_segments = search_res.get_shots()
    
    #no exact match found.
    if len(matched_segments) == 0:
        continue

    #video segment
    video_shot = matched_segments[0]

    #Create a new Video Asset and add it to a timeline.
    timeline.add_inline(VideoAsset(asset_id=video.id, start=video_shot.start, end=video_shot.end))

In [38]:
stream = timeline.generate_stream()
print(stream)
play_stream(stream)

https://stream.videodb.io/v3/published/manifests/b92cb532-0c79-4632-b51a-28143d5d1d62.m3u8


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/b92cb532-0c79-4632-b51a-28143d5d1d62.m3u8'